In [2]:
import os
import zipfile
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
# Download required NLTK data for stopwords
nltk.download('stopwords')

# Initialize stop words and stemmer
stop_words = set(stopwords.words('english'))  # Set of English stop words
stemmer = PorterStemmer()  # Initialize the Porter Stemmer for stemming words

# Function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation and non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenize, remove stop words, and stem the words
    tokens = [stemmer.stem(word) for word in text.split() if word not in stop_words and len(word) > 2]
    return ' '.join(tokens)  # Join tokens back into a single string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Load and preprocess emails from a folder
def load_and_preprocess_emails(folder):
    emails = []  # Initialize an empty list to store emails
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)  # Get full path for the email file
        try:
            # Attempt to read the email with UTF-8 encoding
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()  # Read the content of the email
                processed_text = preprocess_text(text)  # Preprocess the email text
                emails.append(processed_text)  # Append processed text to the list
        except UnicodeDecodeError:
            # Handle potential encoding issues by trying a different encoding
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                text = file.read()
                processed_text = preprocess_text(text)
                emails.append(processed_text)
    return emails  # Return the list of processed emails

In [5]:
# Extract the contents of the zip file
zip_file_path = '/content/FML_DATA.zip'
extracted_folder = '/content/extracted_data'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)  # Extract all files to the specified folder

In [6]:
# Define paths for spam and ham folders
spam_folder = os.path.join(extracted_folder, 'spam')
ham_folder = os.path.join(extracted_folder, 'ham')

# Load emails from both spam and ham folders
spam_emails = load_and_preprocess_emails(spam_folder)  # Load spam emails
ham_emails = load_and_preprocess_emails(ham_folder)    # Load ham emails

In [7]:
# Feature extraction with TF-IDF Vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000)  # Initialize TF-IDF vectorizer
all_emails = spam_emails + ham_emails  # Combine spam and ham emails
X = vectorizer.fit_transform(all_emails)  # Fit and transform the combined emails into TF-IDF features

In [8]:
# Labels: 1 for spam, 0 for ham
y = [1] * len(spam_emails) + [0] * len(ham_emails)

# Split the dataset into training and validation sets, stratifying by labels to maintain balance
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# We are going to train the following models on our training dataset and validate them on our validation dataset:
##1. K-Nearest Neighbours(KNN)
##2. Naive Bayes Classifier
##3. Logistic Regression
##4. Support Vector Machine(SVM)
##5. Perceptron Algorithm

# 1.Building K-Nearest Neighbours from scratch

In [36]:
class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)

    def _predict(self, x):
        distances = np.linalg.norm(self.X_train - x, axis=1)
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = np.bincount(k_nearest_labels).argmax()
        return most_common


In [39]:
# We take k to be 5
k = 5 # We can do cross-validation, but considering the computational time required, we will go with common k value
knn = KNN(k=k)
knn.fit(X_train.toarray(), y_train)
predictions = knn.predict(X_val.toarray())
accuracy = accuracy_score(y_val, predictions)


In [40]:
print(accuracy)

0.9619686800894854


# KNN model has achieved an accuracy score of 96.2% on its validation set.

#2. Building Naive-Bayes Classifier

In [12]:
class NaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.parameters = {}
        for c in self.classes:
            X_c = X[y==c]
            self.parameters[c] = {
                'mean': X_c.mean(axis=0),
                'var': X_c.var(axis=0),
                'prior': len(X_c) / len(X)
            }

    def predict(self, X):
        return [self._predict(x) for x in X]

    def _predict(self, x):
        posteriors = []
        for c in self.classes:
            prior = self.parameters[c]['prior']
            # Calculate likelihood for the entire document
            likelihood = np.prod(self._pdf(c, x))
            posterior = likelihood * prior
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]

    def _pdf(self, c, x):
        mean = self.parameters[c]['mean']
        var = self.parameters[c]['var']
        numerator = np.exp(-(x-mean)**2/(2*var))
        denominator = np.sqrt(2*np.pi*var)
        return numerator / denominator

In [13]:
naive_bayes = NaiveBayes()
naive_bayes.fit(X_train.toarray(), y_train)

In [14]:
prediction_nb = naive_bayes.predict(X_val.toarray())
accuracy_score(prediction_nb, y_val)

<ipython-input-12-c8f75266acb2>:29: RuntimeWarning: invalid value encountered in divide
  numerator = np.exp(-(x-mean)**2/(2*var))
<ipython-input-12-c8f75266acb2>:29: RuntimeWarning: divide by zero encountered in divide
  numerator = np.exp(-(x-mean)**2/(2*var))
<ipython-input-12-c8f75266acb2>:31: RuntimeWarning: invalid value encountered in divide
  return numerator / denominator


0.46308724832214765

# Naive Bayes classifier has achieved an accuracy score of 46.3% on its validation set.

# 3. Building Logistic Regression Model from scratch

In [15]:
class LogisticRegression:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._sigmoid(linear_model)

            # Update weights and bias
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        return [1 if i>0.5 else 0 for i in y_predicted]

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

In [16]:
# Train Logistic Regression model
lr = LogisticRegression(learning_rate=0.1, n_iters=1000)
lr.fit(X_train.toarray(), y_train) # Convert X_train to dense array

In [17]:
# Make predictions
y_pred = lr.predict(X_val.toarray())

In [18]:
# Evaluate model
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.894


# Logistic Regression model has achieved an accuracy score of 89.4% on its validation set.

# 4. Building SVM from scratch

In [19]:
from sklearn import svm

class SupportVectorMachine:
    def __init__(self, kernel='linear', C=1):
        self.kernel = kernel
        self.C = C
        self.model = svm.SVC(kernel=kernel, C=C)

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)


In [22]:
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
for k in kernel:
  svm_model = SupportVectorMachine(kernel=k, C=1)
  svm_model.fit(X_train.toarray(), y_train)
  # Make predictions
  y_pred = svm_model.predict(X_val.toarray())
  # Evaluate model
  accuracy = accuracy_score(y_val, y_pred)
  print(f"Accuracy: {accuracy:.3f}for SVM having kernel {k}")

Accuracy: 0.974for SVM having kernel linear
Accuracy: 0.967for SVM having kernel poly
Accuracy: 0.979for SVM having kernel rbf
Accuracy: 0.970for SVM having kernel sigmoid


# We can observe that SVM having rbf kernel gives the maximum accuracy of 97.9% on its validation set.

In [32]:
svm_rbf = SupportVectorMachine(kernel='rbf', C=1)
svm_rbf.fit(X_train.toarray(), y_train)

# 5. Building Perceptron Algorithm from scratch

In [29]:
import scipy

class Perceptron:
    def __init__(self, lr=0.01, n_iters=250):
        self.lr = lr
        self.n_iters = n_iters
        self.wts = None
        self.bias = None
        self.activation_fn = self.activation_func

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.wts = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                #If x_i is sparse, convert it to dense before dot product
                if isinstance(x_i,(np.matrix, np.ndarray)):
                  x_i = x_i.toarray()[0] if x_i.shape[0] == 1 else x_i

                linear_output = np.dot(x_i, self.wts) + self.bias
                # The linear_output is already a scalar or a dense array, no need to call toarray()
                y_predicted = self.activation_func(linear_output)

                #if x_i is sparse, convert it to dense for update
                if isinstance(x_i,(np.matrix, np.ndarray)):
                  x_i = x_i.toarray()[0] if x_i.shape[0] == 1 else x_i

                update = self.lr * (y[idx] - y_predicted)
                self.wts += update * x_i
                self.bias += update

    def predict(self, X):
        # Convert X to dense array before prediction if it's sparse
        if isinstance(X, (scipy.sparse.csr_matrix, scipy.sparse.csc_matrix)):
            X = X.toarray()  # Convert to dense array if sparse

        linear_output = np.dot(X, self.wts) + self.bias
        y_pred = self.activation_fn(linear_output)
        return y_pred

    def activation_func(self, x):
        return np.where(x >= 0, 1, 0)

In [31]:
percept = Perceptron(lr=0.5, n_iters=250)
percept.fit(X_train.toarray(), y_train)

y_pred = percept.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy:} for perceptron algorithm")

Accuracy: 0.9642058165548099 for perceptron algorithm


# Perceptron algorithm is giving an accuracy of around 96.4%

Hence, here is the compilation of accuracies of all the algorithms that we have trained and validated till now:
1. K-Nearest Neighbours - 96.2
2. Naive Bayes Classifier - 46.3%
2. Logistic Regression  - 89.4
3. SVM ( RBF Kernel)    - 97.9
4. Perceptron           - 96.4

So, we will build our final working fnction using SVM's RBF kernel, to read mails in "test" folder, and make predictions accordingly as "spam" or "ham(non-spam)

In [ ]:
import csv

def predict_test_emails(svm_model, vectorizer):
    test_folder = 'test' # Modify the 'test' folder address
    test_emails = []

    # Read emails from test folder
    for filename in os.listdir(test_folder):
        if filename.startswith('email') and filename.endswith('.txt'):
            file_path = os.path.join(test_folder, filename)
            try:
                # Attempt to read the email with UTF-8 encoding
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()  # Read the content of the email
                    processed_text = preprocess_text(text)  # Preprocess the email text
                    test_emails.append(processed_text)  # Append processed text to the list
            except UnicodeDecodeError:
                # Handle potential encoding issues by trying a different encoding
                with open(file_path, 'r', encoding='ISO-8859-1') as file:
                    text = file.read()
                    processed_text = preprocess_text(text)
                    test_emails.append(processed_text)

    # vectorize test emails(tfidf vectorizer is fitted and performaed transformation already)
    test_emails_vectorized = vectorizer.transform(test_emails)

    # Predict spam/non-spam labels
    predictions = svm_model.predict(test_emails_vectorized)

    # Save predictions to file
    with open('predictions.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Filename', 'Prediction'])  # Write header row
        for i, prediction in enumerate(predictions):
            filename = os.listdir(test_folder)[i]  # Get filename
            writer.writerow([filename, prediction])

svm_model = svm_rbf
vectorizer = vectorizer
# Calling the prediction function
predict_test_emails(svm_model, vectorizer)